# State Monad

## Outline

* Incentive for the State monad

* The State monad
  - Definition of the State monad
  - MonadState type class
  - Simple example
<br><br>
* State monad examples
  - State monad with IO actions

In this lesson, we will learn about the State monad type and how you can use it.

## Incentive for State monad

We talked in the previous two lectures how you can use the Reader and Writer Monad if you have to read from a environment variable or write to it.

If you need to perform both operations you can use the State Monad. Let's first look at an example where we perform read and write operations without the State monad.

We will create the Tic-Tac-Toe game (https://en.wikipedia.org/wiki/Tic-tac-toe) that will use the **System.Random** to randomly pick the X and O choices.

In [ ]:
import System.Random (mkStdGen, Random(randomR), StdGen)

data Player = XPlayer | OPlayer deriving Eq
data Choice = Empty | X | O deriving Eq

data GameState = GameState
  { currentBoard :: [Choice]
  , currentPlayer :: Player
  , generator :: StdGen
  }

main :: IO ()
main = do
    putStrLn "Game results:"
    let gen = mkStdGen 1
        initState = GameState
                      [Empty | boardInd <- [1..9]]
                      XPlayer
                      gen
    playGame initState
    
playGame :: GameState -> IO ()
playGame gs = do
    let freeFields = getFreeFields gs
    if length freeFields /= 0
    then do
        let player = currentPlayer gs
            board = currentBoard gs
            gen = generator gs
            (choiceInd, gen') = randomR (0, length freeFields - 1) gen
            choice = (freeFields !! choiceInd) + 1
            newGameState = GameState
                            (if player == XPlayer
                            then [if ind /= choice then board !! (ind-1) else X | ind <- [1..9]]
                            else [if ind /= choice then board !! (ind-1) else O | ind <- [1..9]])
                            (nextPlayer player)
                            gen'
        playGame newGameState
    else do
        printBoard gs

getFreeFields :: GameState -> [Int]
getFreeFields gs = [ind | ind <- [0..8], board !! ind == Empty]
    where board = currentBoard gs

nextPlayer :: Player -> Player
nextPlayer XPlayer = OPlayer
nextPlayer OPlayer = XPlayer

printBoard :: GameState -> IO ()
printBoard gs = do
    let board = currentBoard gs
    let stateToString st = case st of
                             Empty -> "-"
                             X -> "X"
                             O -> "O"
        printInd ind = stateToString $ board !! ind
    mapM_ putStr [printInd 0,"|", printInd 1,"|", printInd 2, "\n"]
    putStrLn "-----"
    mapM_ putStr [printInd 3,"|", printInd 4,"|", printInd 5, "\n"]
    putStrLn "-----"
    mapM_ putStr [printInd 6,"|", printInd 7,"|", printInd 8, "\n"]

main

In the code above we define the **GameState** variable that holds data for the current state of the game. In the **main** function we create the initial state.

In the **playGame** function we use the **GameState** variable to create a new move and update the variable before we recursivly call this function again or print the game results.

This is a simple game but if it would become more complex you would need to pass around a lot of data. For this reason the **State monad** was created.

## The State monad

### Definition of the State monad

Let's first look at the user freindly definition of the **State** type and then the actuall Haskell2010 definition that uses transformers.

The definition of the **State** type can be written as:
```haskell
newtype State s a = State { runState :: s -> (a, s) }
```

We see that the `State` data constructor holds a function that can be accessed with the name `runState`.

It takes in a state and returns a tuple that contains a variable of type `a` and another state variable.

We can now create a Monad instance for `State s` and not just `State`.

This means the type of our state will remain the same as we compose our function with `(>>=)`.
```haskell
(>>=) :: State s a        ->
         (a -> State s b) ->
         State s b
```

The monad instance for `State s` would be then:
```haskell
instance Monad (State s) where
    return a = State $ \s -> (a, s)
    m >>= k = State $ \s -> let (a, s') = runState m s
                            in runState (k a) s'
```

The actual Haskell2010 definition of the `State` type is defined in terms of the `StateT` monad transformer:
```haskell
type State s = StateT s Identity
```
We will learn more about it when talk about Monad transformers.

### MonadState type class

The `MonadState` type class defines functions wich you can use to work with a state monad. It is defined in the **Control.Monad.State** module.

Two functions that are usefull for working with the state monad are `get` and `put` which can get or set the state variable of the state moand.

```haskell
class Monad m => MonadState s m | m -> s where
    get :: m s
    get = state (\s -> (s, s))

    put :: s -> m ()
    put s = state (\_ -> ((), s))

    state :: (s -> (a, s)) -> m a
    state f = do
      s <- get
      let ~(a, s') = f s
      put s'
      return a
    {-# MINIMAL state | get, put #-}
```

To be able to declare a type class with two types you need to use the *MultiParamTypeClasses* language pragma.

If the type `m` which in reality for us will be `(State s)`, is defiinng the type `s` we can state this dependency with the `| m -> s` statement.

To be able to use such dependency statements in type class declarations we need to include the *FunctionalDependencies* language pragma.

Form the module that contains the `MonadState` type class we can also use the `modify` and `gets` functions.
```haskell
modify :: MonadState s m => (s -> s) -> m ()
modify f = state (\s -> ((), f s))

gets :: MonadState s m => (s -> a) -> m a
gets f = do
    s <- get
    return (f s)
```

The `modify` function takes in a function and updates the state monad such that is applies the function to the state variable.

The `gets` function takes in a function, applies it to the state variable and writes the result in the type `a` variable of the state monad.

### Simple example

Let's write an example of a State monad that helps us compute the first n fibonacci numbers.

In [ ]:
import Control.Monad.State (MonadState(put, get), runState, State)

type FibNums = [Int]

main :: IO ()
main = do
    putStrLn "How many fibonacci numbers do you want to see:"
    n <- (read <$> getLine) :: IO Int
    let initialState = [n,0,1]
    process initialState

process :: FibNums -> IO ()
process st = do
    let (finished, newSt) = runState addFib st
    if finished 
    then do
      putStrLn "Fibonacci numbers are:"
      print $ tail newSt
    else process newSt

addFib :: State FibNums Bool
addFib = do
    st <- get
    let newSt = st ++ [last st + last (init st)]
        finished = length (tail newSt) == head newSt 
    put newSt
    return finished

main

First ask the user for a number and then run the **process** function that we provide with the initial state.

Out initial state is a list that hold the number of fibonacci elements we want and the first two fibonacci numbers.

In the **process** function we start the State monad **addFib** with the provided state and then check if the list is complete. If not we recursively call **process** again. 

The **addFib** State monad function first gets the state, then it updates the state and returns a Bool that inidicates if we computed the desired length.

## State monad examples

Let's look now at our initial example and how we can re-write it with the use of the State monad.

Just to have an overview we will write out all also the code that is duplicated form the initial example.

In [ ]:
import Control.Monad.State (MonadState(get, put), runState, State)
import System.Random (mkStdGen, Random(randomR), StdGen)

data Player = XPlayer | OPlayer deriving Eq
data Choice = Empty | X | O deriving Eq

data GameState = GameState
  { currentBoard :: [Choice]
  , currentPlayer :: Player
  , generator :: StdGen
  }

main :: IO ()
main = do
  let gen = mkStdGen 1
      initState = GameState
                  [Empty | boardInd <- [1..9]]
                  XPlayer
                  gen
  playGame initState

playGame :: GameState -> IO ()
playGame gs = do
  let (gameFinished, newGS) = runState resolveTurn gs
  if gameFinished 
  then do
    putStrLn "Game results:"
    printBoard newGS
  else playGame newGS

resolveTurn :: State GameState Bool
resolveTurn = do
  choice <- chooseRandomMove
  applyMove choice
  isGameDone

chooseRandomMove :: State GameState Int
chooseRandomMove = do
  gs <- get
  let board = currentBoard gs
      openSpots = [ind | ind <- [0..8], board !! ind == Empty]
      gen = generator gs
  let (i, gen') = randomR (0, length openSpots - 1) gen
  put $ gs { generator = gen' }
  return $ (openSpots !! i) + 1

applyMove :: Int -> State GameState ()
applyMove choice = do
  gs <- get
  let player = currentPlayer gs
      board = currentBoard gs
      newBoard = if player == XPlayer
                 then [if ind /= choice then board !! (ind-1) else X | ind <- [1..9]]
                 else [if ind /= choice then board !! (ind-1) else O | ind <- [1..9]]
  put $ gs { currentPlayer = nextPlayer player, currentBoard = newBoard }

nextPlayer :: Player -> Player
nextPlayer XPlayer = OPlayer
nextPlayer OPlayer = XPlayer

isGameDone :: State GameState Bool
isGameDone = do
  gs <- get
  let board = currentBoard gs
      openSpots = [ind | ind <- [0..8], board !! ind == Empty]
  return $ length openSpots == 0

printBoard :: GameState -> IO ()
printBoard gs = do
    let board = currentBoard gs
    let stateToString st = case st of
                             Empty -> "-"
                             X -> "X"
                             O -> "O"
        printInd ind = stateToString $ board !! ind
    mapM_ putStr [printInd 0,"|", printInd 1,"|", printInd 2, "\n"]
    putStrLn "-----"
    mapM_ putStr [printInd 3,"|", printInd 4,"|", printInd 5, "\n"]
    putStrLn "-----"
    mapM_ putStr [printInd 6,"|", printInd 7,"|", printInd 8, "\n"]

main

Here we use the same **GameState** variable as before that now represents the state of our State monad.

In the **playGame** function we start the **resolveTurn** State monad and provide it the initial state.

This monad calls then three other State monads that update the state accordingly and the final return parameter tells us weather the game is finished.

## Recap

In this lesson we've discussed:

- the motivation for introducing the State monad type 

- the definition of the State monad type and a simple example

- examples that use the State monad type and functions that work with it